In [ ]:
import RS_SMA100B

ip = '' # Set IP address of the device
pulse_freq = 0 # Set frequency of the signal to send to the resonator
amplitude = -25 # Set the amplitude of the signal in dBm
pulse_time = 2e-6 # 2 microsec
sGen = RS_SMA100B.SMA100B(ip)

sGen.reset()
sGen.clear()
sGen.RF_freq(pulse_freq) # in Hz
sGen.RF_lvl_ampl(amplitude)
sGen.pul_gen_params(period = pulse_time, delay = 0, width = pulse_time)
sGen.pul_gen_mode('SING')
sGen.pul_exe_sing_trig() # DA TESTARE

sGen.pul_state(1)
sGen.RF_state(1)

In [ ]:
from Card_Class import *

PXIe5170R('DeviceName')
pxie.available

# configure parameters
pxie.voltage_range = # ??
pxie.sample_rate = 2 * pulse_freq # f_sample > 2*f_max
pxie.coupling = 'AC' # or DC????
pxie.num_pts = # ??
pxie.num_records = 1
pxie.ref_position = 50.0

# configure the acquisition
print(pxie.acquisition())